***카테고리 딕션어리***

In [ ]:
main_cat_dict = {
    '스킨케어' : '10000010001',
    '바디케어' : '10000010003',
    '헤어케어' : '10000010004',
    '미용소품' : '10000010006',
    '남성' : '10000010007',
    '마스크팩' : '10000010009',
    '클렌징' : '10000010010',
    '선케어' : '10000010011',
    '여성/위생용품': '10000020004',
}

sub_cat_dict = {
    # 스킨케어
    '토너/로션/올인원' : '100000100010008',
    '에센스/크림' : '100000100010009',
    '미스트/오일' : '100000100010010',
    
    # 바디케어
    '샤워/입욕' : '100000100030005',
    '로션/오일' : '100000100030014',
    '바디미스트' : '100000100030015',
    
    # 헤어케어
    '트리트먼트/팩' : '100000100040007',
    '샴푸/린스' : '100000100040008',

    # 미용소품
    '스킨케어소품' : '100000100060006',
    
    # 남성
    '스킨케어' : '100000100070007',
    '헤어케어' : '100000100070009',
    '쉐이빙' : '100000100070010',
    '바디케어' : '100000100070011',
    '향수/매너용품' : '100000100070017',
    
    # 마스크팩
    '시트팩' : '100000100090001',
    '페이셜팩' : '100000100090002',
    '코팩/패치' : '100000100090003',
    '패드' : '100000100090004',
    
    # 클렌징
    '클렌징폼/젤' : '100000100100001',
    '오일/워터/리무버' : '100000100100002',
    '필랭/패드' : '100000100100003',
    
    # 선케어
    '선블록' : '100000100110001',
    '태닝/애프터선' : '100000100110002',
    
    # 여성/위생용품
    '여성청결제' : '100000200040002'
    
}


# 닥터지 카테고리 
category_dict_01 = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009'  # 에센스/크림
    ]
}


***상품 넘버 가져오기***

***brand_dict, final_dict만 활용***

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import json
import time

category_dict = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009'  # 에센스/크림
    ],
    '10000010007': [ # 남성
        '100000100070007', # 스킨케어
        '100000100070009', # 헤어케어
        '100000100070010', # 쉐이빙
        '100000100070011', # 바디케어
        '100000100070017' # 향수/매너용품
    ],
    '10000010010' : [ # 클렌징
        '100000100100001', # 클렌징폼/젤
        '100000100100002', # 오일/워터/리무버
        '100000100100003' # 필링/패드
    ],
    '10000010009' : [ # 마스크팩
        '100000100090001', # 시트팩
        '100000100090002', # 페이셜팩
        '100000100090003', # 코팩/패치
        '100000100090004', # 패드
    ]
}

def bring_brand_category_dictionary():
    brand_list = ['닥터지', '라운드랩', '식물나라'] # 브랜드 이름 추가 및 
    brand_dict = {}
    final_dict = {}
    for brand in brand_list:
        for main, subs in category_dict.items():
            for sub in subs:
                try: # try/except를 하는 이유는 category_dict에 있는 메인 혹은 서브 카테고리가 해당 브랜드에 없을 수도 있기 때문에
                     # 없는 경우 넘어가주도록 합니다. 
                    startCount = 0
                    while True:
                        base_url = 'https://www.oliveyoung.co.kr/store/search/NewMainSearchApi.do?'
                        params_01 = {
                            'query' : brand, # 브랜드별
                            'collection': 'OLIVE_GOODS,OLIVE_PLAN,OLIVE_EVENT,OLIVE_BRAND',
                            'listnum': '36',      
                            'startCount' : startCount, # listnum이 36이 넘어가는 경우 그 다음 페이지도 가져올 수 있도록
                            'sort' : 'RNK/DESC',
                            'goods_sort' : 'WEIGHT/DESC,RNK/DESC',
                            'disPlayCateId' : main,
                            'cateId': main, # main category
                            'cateId2': sub, # sub category
                            'typeChk' : 'thum',
                            'quickYn' : 'N'
                        }

                        headers = { 'X-Requested-With' : 'XMLHttpRequest'}
                        res = requests.get(base_url, headers = headers, params=params_01)
                        crawled_data = json.loads(res.json())

                        goodsNo_list = list() # 다음페이지의 상품 리스트가 있는지 없는지 확인하기 위한 리스트 
                        tmp_dict = {}
                        for data in crawled_data['Data'][0]['Result']: # 'Result' key부분에 상품 번호가 있습니다. 
                            goodsNo_list.append(data['GOODS_NO']) 
                        brand_dict[sub] = goodsNo_list # brand_dict에 먼저 넣어주고
                        final_dict[brand] = brand_dict # 다음에 final_dict에 넣어줍니다. 

                        startCount += int(params_01['listnum']) # startCount를 listnum만큼 넣어주고 다음 페이지로 이동합니다
                        if len(goodsNo_list) <= int(params_01['listnum']): # 다음 페이지에 listnum만큼 없는 경우 break를 하고 다음으로 넘어갑니다.
                            break
                except: # 메인 혹은 서브 카테고리가 없는 경우 넘어가주기 
                    continue

                    time.sleep(0.3) # 혹시나 블락을 당할 경우를 고려하여 time.sleep()을 줍니다. 

    return final_dict

bring_brand_category_dictionary()

***sub_dict, main_dict, final_dict 활용***

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import json
import time

category_dict = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009'  # 에센스/크림
    ],
    '10000010007': [ # 남성
        '100000100070007', # 스킨케어
        '100000100070009', # 헤어케어
        '100000100070010', # 쉐이빙
        '100000100070011', # 바디케어
        '100000100070017' # 향수/매너용품
    ],
    '10000010010' : [ # 클렌징
        '100000100100001', # 클렌징폼/젤
        '100000100100002', # 오일/워터/리무버
        '100000100100003' # 필링/패드
    ],
    '10000010009' : [ # 마스크팩
        '100000100090001', # 시트팩
        '100000100090002', # 페이셜팩
        '100000100090003', # 코팩/패치
        '100000100090004', # 패드
    ]
}

def bring_brand_category_dictionary():
    brand_list = ['닥터지', '라운드랩', '식물나라'] # 브랜드 이름 추가 및 
    sub_dict = {}
    main_dict = {}
    final_dict = {}
    for brand in brand_list:
        for main, subs in category_dict.items():
            for sub in subs:
                try: # try/except를 하는 이유는 category_dict에 있는 메인 혹은 서브 카테고리가 해당 브랜드에 없을 수도 있기 때문에
                     # 없는 경우 넘어가주도록 합니다. 
                    startCount = 0
                    while True:
                        base_url = 'https://www.oliveyoung.co.kr/store/search/NewMainSearchApi.do?'
                        params_01 = {
                            'query' : brand, # 브랜드별
                            'collection': 'OLIVE_GOODS,OLIVE_PLAN,OLIVE_EVENT,OLIVE_BRAND',
                            'listnum': '36',      
                            'startCount' : startCount, # listnum이 36이 넘어가는 경우 그 다음 페이지도 가져올 수 있도록
                            'sort' : 'RNK/DESC',
                            'goods_sort' : 'WEIGHT/DESC,RNK/DESC',
                            'disPlayCateId' : main,
                            'cateId': main, # main category
                            'cateId2': sub, # sub category
                            'typeChk' : 'thum',
                            'quickYn' : 'N'
                        }

                        headers = { 'X-Requested-With' : 'XMLHttpRequest'}
                        res = requests.get(base_url, headers = headers, params=params_01)
                        crawled_data = json.loads(res.json())

                        goodsNo_list = list() # 다음페이지의 상품 리스트가 있는지 없는지 확인하기 위한 리스트 
                        tmp_dict = {}
                        for data in crawled_data['Data'][0]['Result']: # 'Result' key부분에 상품 번호가 있습니다. 
                            goodsNo_list.append(data['GOODS_NO']) 
                        sub_dict[sub] = goodsNo_list # sub_dict에 먼저 넣어주고
                        main_dict[main] = sub_dict # 그 다음에 main_dict에 넣어주고
                        final_dict[brand] = main_dict # 마지막으로 final_dict에 넣어줍니다. 

                        startCount += int(params_01['listnum']) # startCount를 listnum만큼 넣어주고 다음 페이지로 이동합니다
                        if len(goodsNo_list) <= int(params_01['listnum']): # 다음 페이지에 listnum만큼 없는 경우 break를 하고 다음으로 넘어갑니다.
                            break
                except: # 메인 혹은 서브 카테고리가 없는 경우 넘어가주기 
                    continue

                    time.sleep(0.3) # 혹시나 블락을 당할 경우를 고려하여 time.sleep()을 줍니다. 

    return final_dict

bring_brand_category_dictionary()

{'닥터지': {'10000010001': {'100000100010008': ['A000000173724',
    'A000000150467',
    'A000000173745',
    'A000000163512',
    'A000000173723'],
   '100000100010009': ['A000000166586',
    'A000000180461',
    'A000000180476',
    'A000000180479',
    'A000000166590',
    'A000000145579',
    'A000000166595',
    'A000000138097',
    'A000000163496',
    'A000000168742',
    'A000000156838',
    'A000000144843',
    'A000000156846',
    'A000000143718'],
   '100000100070007': ['A000000175945',
    'A000000175946',
    'A000000163494',
    'A000000183230',
    'A000000183229',
    'A000000163496',
    'A000000183227',
    'A000000163493'],
   '100000100070009': ['A000000163563'],
   '100000100070011': ['A000000171625', 'A000000153504'],
   '100000100070017': ['A000000153504'],
   '100000100100001': ['A000000151915',
    'A000000179898',
    'A000000173139',
    'A000000179892',
    'A000000151914'],
   '100000100100002': ['A000000152532',
    'A000000152348',
    'A000000178794',
    

***리뷰 뽑아오는 코드***

In [126]:
import time


review_dict = {
    'goodsNo' : [],
    'gdasCont' : [],
    'goodsNm' : [],
    'dispRegDate' : [],
    
}
# review_list = []

base_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?'
headers = {'X-Requested-With' : 'XMLHttpRequest'}
params = {
    'goodsNo' : 'A000000165683', # 상품 사전 가져오기 ==> 여기 부분 바꾸고
    'gdasSort': '02', # 그대로 놔두기
    'itemNo' : 'all_search',
    'pageIdx' : '1', # 페이지 반복문 필요 ==> 여기 부분 바꾸고 
    'cTypeLength' : '0'
}

res = requests.get(base_url, headers = headers, params=params)
crawled_data = json.loads(res.text)
for i in range(len(crawled_data['gdasList'])):
#     review_list.append(crawled_data['gdasList'][i]['gdasCont'])
    review_dict['goodsNo'].append(crawled_data['gdasList'][i]['goodsNo'])
    review_dict['gdasCont'].append(crawled_data['gdasList'][i]['gdasCont'])
    review_dict['goodsNm'].append(crawled_data['gdasList'][i]['goodsNm'])
    review_dict['dispRegDate'].append(crawled_data['gdasList'][i]['dispRegDate'])

# pd.DataFrame(review_dict)
crawled_data['gdasList'][0]
        

{'sysRegDtime': None,
 'sysRegrId': None,
 'sysModDtime': None,
 'sysModrId': None,
 'rowsPerPage': 0,
 'pageIdx': 1,
 'pageCalc': 0,
 'systemDefaultLanguage': '',
 'dbLocaleLanguage': 'ko',
 'state': None,
 'defaultCntryCd': '',
 'sysRegrNm': None,
 'sysModrNm': None,
 'fieldNames': None,
 'texts': None,
 'widths': None,
 'excelTitle': None,
 'excelYn': None,
 'excelCurPage': None,
 'excelCnt': None,
 'excelTcnt': None,
 'viewCode': None,
 'gdasSeq': 19106054,
 'goodsNo': 'A000000165683',
 'ordNo': '12023042513121007100100008',
 'ordGoodsSeq': '1',
 'lgcGoodsNo': None,
 'titNm': None,
 'gdasScrVal': 8,
 'prcScrVal': None,
 'qualScrVal': None,
 'dlvScrVal': None,
 'gdasCont': '자극없이 순하게 모공관리 그리고 보습감 까지 필요하다면 이 제품이 딱인것 같아요.\r\n자극이 없어서 노페물이 잘 제거될지 걱정되었는데 깔끔하게 정돈되어서 만족스럽네요.!',
 'purMbrYn': 'Y',
 'qryCnt': 0,
 'dispStatCd': '10',
 'dispProcDtime': 1682420447000,
 'dispDspsId': None,
 'confStatCd': '10',
 'confProcDtime': 1682420447000,
 'confDspsId': None,
 'mbrNo': 'M0000018990098',
 'phot